# viz_gradient_3d\n\nNotebook para construir intuición de Gradient Descent con una superficie 3D (Plotly) y un slider de `learning_rate` (ipywidgets).\n\nRecomendación: ejecutar en JupyterLab.

In [ ]:
import ipywidgets as widgets
import numpy as np
import plotly.graph_objects as go
from IPython.display import display


def f(x, y):
    return x**2 + 4 * y**2


def grad_f(x, y):
    return np.array([2 * x, 8 * y])


def run_gd(lr=0.1, steps=20, x0=2.5, y0=2.5):
    x, y = float(x0), float(y0)
    path = [(x, y, f(x, y))]
    for _ in range(int(steps)):
        g = grad_f(x, y)
        x = x - lr * g[0]
        y = y - lr * g[1]
        path.append((x, y, f(x, y)))
    return np.array(path)


def make_figure(lr=0.1, steps=20):
    grid = np.linspace(-3, 3, 80)
    X, Y = np.meshgrid(grid, grid)
    Z = f(X, Y)

    path = run_gd(lr=lr, steps=steps)

    fig = go.Figure()
    fig.add_trace(
        go.Surface(x=X, y=Y, z=Z, opacity=0.85, colorscale="Viridis", showscale=False)
    )
    fig.add_trace(
        go.Scatter3d(
            x=path[:, 0],
            y=path[:, 1],
            z=path[:, 2],
            mode="lines+markers",
            line={"color": "red", "width": 6},
            marker={"size": 4},
        )
    )

    fig.update_layout(
        title=f"Gradient Descent: lr={lr}",
        scene={"xaxis_title": "x", "yaxis_title": "y", "zaxis_title": "f(x,y)"},
        height=650,
        margin={"l": 0, "r": 0, "t": 40, "b": 0},
    )
    return fig


lr_slider = widgets.FloatLogSlider(
    value=0.1, base=10, min=-3, max=0, step=0.05, description="lr"
)
steps_slider = widgets.IntSlider(value=20, min=5, max=60, step=1, description="steps")
out = widgets.Output()


def refresh(*args):
    with out:
        out.clear_output(wait=True)
        fig = make_figure(lr=float(lr_slider.value), steps=int(steps_slider.value))
        fig.show()


lr_slider.observe(refresh, names="value")
steps_slider.observe(refresh, names="value")

display(widgets.VBox([widgets.HBox([lr_slider, steps_slider]), out]))
refresh()